# Molecular Property Prediction by ChemProp

data: https://ogb.stanford.edu/docs/graphprop/

In [1]:
import io
import sys
import zipfile

import pandas as pd
import requests
import torch
import torch.nn as nn
from ogb.graphproppred import Evaluator, PygGraphPropPredDataset
from sklearn.metrics import accuracy_score, roc_auc_score
from torch_geometric.data import DataLoader
from tqdm import tqdm

from pyg_chemprop import DMPNNEncoder, RevIndexedDataset
from pyg_chemprop_utils import FeatureScaler, NoamLR, initialize_weights, smiles2data

Using backend: pytorch


In [2]:
data_dir = "./data"

## preparation

In [3]:
pyg_dataset = PygGraphPropPredDataset(name="ogbg-molhiv", root=data_dir)
split_idx = pyg_dataset.get_idx_split()
pyg_dataset.task_type

'binary classification'

In [4]:
dataset = RevIndexedDataset(pyg_dataset)

100%|██████████| 41127/41127 [00:47<00:00, 872.14it/s] 


In [5]:
batch_size = 50

train_data = torch.utils.data.Subset(dataset, split_idx["train"])
valid_data = torch.utils.data.Subset(dataset, split_idx["valid"])
test_data = torch.utils.data.Subset(dataset, split_idx["test"])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [6]:
def train(config, loader, device=torch.device("cpu")):
    criterion = config["loss"]
    model = config["model"]
    optimizer = config["optimizer"]
    scheduler = config["scheduler"]

    model = model.to(device)
    model.train()
    for batch in tqdm(loader, total=len(loader)):
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        scheduler.step()

In [7]:
def make_prediction(config, loader, device=torch.device("cpu")):
    model = config["model"]

    model = model.to(device)
    model.eval()
    y_pred = []
    y_true = []
    for batch in tqdm(loader, total=len(loader)):
        batch = batch.to(device)
        with torch.no_grad():
            batch_preds = torch.sigmoid(model(batch))
        y_pred.extend(batch_preds)
        y_true.extend(batch.y)
    return torch.stack(y_pred).cpu(), torch.stack(y_true).cpu()

## run test (cpu)

In [8]:
num_epochs = 3
hidden_size = 300
depth = 3
out_dim = 1

In [9]:
head = nn.Sequential(
    nn.Linear(hidden_size, hidden_size, bias=True),
    nn.ReLU(),
    nn.Linear(hidden_size, out_dim, bias=True),
)
model = nn.Sequential(
    DMPNNEncoder(
        hidden_size,
        dataset.num_node_features,
        dataset.num_edge_features,
        depth,
    ),
    head,
)
initialize_weights(model)

In [10]:
dataset.num_node_features, dataset.num_edge_features

(9, 3)

In [11]:
model

Sequential(
  (0): DMPNNEncoder(
    (act_func): ReLU()
    (W1): Linear(in_features=12, out_features=300, bias=False)
    (W2): Linear(in_features=300, out_features=300, bias=False)
    (W3): Linear(in_features=309, out_features=300, bias=True)
  )
  (1): Sequential(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): ReLU()
    (2): Linear(in_features=300, out_features=1, bias=True)
  )
)

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=num_epochs
)

In [13]:
config = {
    "loss": criterion,
    "model": model,
    "optimizer": optimizer,
    "scheduler": scheduler,
}

In [14]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}", file=sys.stderr)
    train(config, train_loader)
    y_pred, y_true = make_prediction(config, valid_loader)
    auc = roc_auc_score(y_true, y_pred)
    acc = accuracy_score(y_true, (y_pred > 0.5).int())
    print(f"val auc={auc:.6} acc={acc:.6}", file=sys.stderr)

Epoch 1
100%|██████████| 83/83 [00:01<00:00, 45.67it/s]
val auc=0.679983 acc=0.980306
Epoch 2
100%|██████████| 83/83 [00:01<00:00, 45.09it/s]
val auc=0.737526 acc=0.980306
Epoch 3
100%|██████████| 83/83 [00:01<00:00, 43.89it/s]
val auc=0.723903 acc=0.980306


In [15]:
y_pred, y_true = make_prediction(config, test_loader)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"test auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 83/83 [00:01<00:00, 46.47it/s]
test auc=0.676228 acc=0.968393


## run test (gpu)

In [16]:
seed = 0
torch.manual_seed(seed);

In [17]:
torch.cuda.is_available()

True

In [18]:
cuda = torch.device("cuda")

In [19]:
dev_id = torch.cuda.current_device()
torch.cuda.get_device_name(dev_id)

'A100-PCIE-40GB'

In [20]:
num_epochs = 30
hidden_size = 300
depth = 3
out_dim = 1

In [21]:
rate = 0.0
head = nn.Sequential(
    nn.Dropout(p=rate, inplace=False),
    nn.Linear(hidden_size, hidden_size, bias=True),
    nn.ReLU(),
    nn.Dropout(p=rate, inplace=False),
    nn.Linear(hidden_size, out_dim, bias=True),
)
model = nn.Sequential(
    DMPNNEncoder(
        hidden_size,
        dataset.num_node_features,
        dataset.num_edge_features,
        depth,
    ),
    head,
)
initialize_weights(model)

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=num_epochs
)

In [23]:
config = {
    "loss": criterion,
    "model": model,
    "optimizer": optimizer,
    "scheduler": scheduler,
}

In [24]:
best = {'epoch': -1, 'score': float("-inf")}
outfile = f"{data_dir}/best_snapshot.pth"

In [25]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}", file=sys.stderr)
    train(config, train_loader, device=cuda)
    y_pred, y_true = make_prediction(config, valid_loader, device=cuda)
    auc = roc_auc_score(y_true, y_pred)
    if auc > best['score']:
        print(f"* best-auc {best['score']:.6} ==> {auc:.6}", file=sys.stderr)
        best['score'] = auc
        best['epoch'] = epoch+1
        torch.save(model.state_dict(), outfile)
    acc = accuracy_score(y_true, (y_pred > 0.5).int())
    print(f"val auc={auc:.6} acc={acc:.6}", file=sys.stderr)

Epoch 1
100%|██████████| 83/83 [00:00<00:00, 181.50it/s]
* best-auc -inf ==> 0.639765
val auc=0.639765 acc=0.980306
Epoch 2
100%|██████████| 83/83 [00:00<00:00, 181.22it/s]
* best-auc 0.639765 ==> 0.709512
val auc=0.709512 acc=0.980306
Epoch 3
100%|██████████| 83/83 [00:00<00:00, 177.98it/s]
* best-auc 0.709512 ==> 0.710167
val auc=0.710167 acc=0.980306
Epoch 4
100%|██████████| 83/83 [00:00<00:00, 182.45it/s]
val auc=0.674986 acc=0.980306
Epoch 5
100%|██████████| 83/83 [00:00<00:00, 182.36it/s]
* best-auc 0.710167 ==> 0.766522
val auc=0.766522 acc=0.980306
Epoch 6
100%|██████████| 83/83 [00:00<00:00, 218.42it/s]
val auc=0.740398 acc=0.980306
Epoch 7
100%|██████████| 83/83 [00:00<00:00, 217.65it/s]
val auc=0.730664 acc=0.980549
Epoch 8
100%|██████████| 83/83 [00:00<00:00, 219.77it/s]
val auc=0.706703 acc=0.980306
Epoch 9
100%|██████████| 83/83 [00:00<00:00, 220.65it/s]
val auc=0.714062 acc=0.981522
Epoch 10
100%|██████████| 83/83 [00:00<00:00, 220.77it/s]
val auc=0.73267 acc=0.982008
Ep

In [26]:
print(f"best validation auc = {best['score']} on epoch {best['epoch']}")

best validation auc = 0.7766754850088183 on epoch 26


#### final 

In [27]:
y_pred, y_true = make_prediction(config, test_loader, device=cuda)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"test auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 83/83 [00:00<00:00, 182.67it/s]
test auc=0.734684 acc=0.967177


#### best snapshot

In [28]:
model = nn.Sequential(
    DMPNNEncoder(
        hidden_size,
        dataset.num_node_features,
        dataset.num_edge_features,
        depth,
    ),
    head,
)
model.load_state_dict(torch.load(outfile))

<All keys matched successfully>

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=num_epochs
)

In [30]:
config = {
    "loss": criterion,
    "model": model,
    "optimizer": optimizer,
    "scheduler": scheduler,
}

In [31]:
y_pred, y_true = make_prediction(config, train_loader, device=cuda)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"train auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 659/659 [00:03<00:00, 210.71it/s]
train auc=0.968617 acc=0.979454


In [32]:
y_pred, y_true = make_prediction(config, valid_loader, device=cuda)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"valid auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 83/83 [00:00<00:00, 219.61it/s]
valid auc=0.776675 acc=0.980793


In [33]:
y_pred, y_true = make_prediction(config, test_loader, device=cuda)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"test auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 83/83 [00:00<00:00, 221.63it/s]
test auc=0.735142 acc=0.966691


# Using ChemProp Atom- and Bond- Features

In [34]:
seed = 0
torch.manual_seed(seed);

In [35]:
def get_dataset(df):
    data_list = []
    print("Convert to PyG Objects...", file=sys.stderr)
    for _, row in tqdm(df.iterrows(), total=len(df)):
        smi = row["smiles"]
        data = smiles2data(smi, explicit_h=True)
        data.y = torch.tensor([[row["HIV_active"]]])
        data_list.append(data)
    print("Convert to RevIndexedDataset...", file=sys.stderr)
    return RevIndexedDataset(data_list)

In [36]:
data_url = "http://snap.stanford.edu/ogb/data/graphproppred/csv_mol_download/hiv.zip"

In [37]:
r = requests.get(data_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(data_dir)

In [38]:
df = pd.read_csv(f"{data_dir}/hiv/mapping/mol.csv.gz")
dataset = get_dataset(df)

Convert to PyG Objects...
 85%|████████▌ | 35119/41127 [00:52<00:08, 684.75it/s]RDKit WARNING: [04:49:10] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [04:49:10] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 41127/41127 [01:02<00:00, 659.30it/s]
Convert to RevIndexedDataset...
100%|██████████| 41127/41127 [01:11<00:00, 576.14it/s]


In [39]:
split = {
    "train": pd.read_csv(f"{data_dir}/hiv/split/scaffold/train.csv.gz", header=None),
    "valid": pd.read_csv(f"{data_dir}/hiv/split/scaffold/valid.csv.gz", header=None),
    "test": pd.read_csv(f"{data_dir}/hiv/split/scaffold/test.csv.gz", header=None),
}
train_idx = split["train"].to_numpy().flatten()
valid_idx = split["valid"].to_numpy().flatten()
test_idx = split["test"].to_numpy().flatten()

In [40]:
batch_size = 50

train_raw = torch.utils.data.Subset(dataset, train_idx)
valid_raw = torch.utils.data.Subset(dataset, valid_idx)
test_raw = torch.utils.data.Subset(dataset, test_idx)

scaler = FeatureScaler(targets=["x", "edge_attr"])
train_data = scaler.fit_transform(train_raw)
valid_data = scaler.transform(valid_raw)
test_data = scaler.transform(test_raw)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

100%|██████████| 4113/4113 [00:00<00:00, 27857.72it/s]


In [41]:
cuda = torch.device("cuda")

In [42]:
dev_id = torch.cuda.current_device()
torch.cuda.get_device_name(dev_id)

'A100-PCIE-40GB'

In [43]:
num_epochs = 30
hidden_size = 300
depth = 3
out_dim = 1

In [44]:
rate = 0.0
head = nn.Sequential(
    nn.Dropout(p=rate, inplace=False),
    nn.Linear(hidden_size, hidden_size, bias=True),
    nn.ReLU(),
    nn.Dropout(p=rate, inplace=False),
    nn.Linear(hidden_size, out_dim, bias=True),
)
model = nn.Sequential(
    DMPNNEncoder(
        hidden_size,
        dataset.num_node_features,
        dataset.num_edge_features,
        depth,
    ),
    head,
)
initialize_weights(model)

In [45]:
model

Sequential(
  (0): DMPNNEncoder(
    (act_func): ReLU()
    (W1): Linear(in_features=147, out_features=300, bias=False)
    (W2): Linear(in_features=300, out_features=300, bias=False)
    (W3): Linear(in_features=433, out_features=300, bias=True)
  )
  (1): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): Linear(in_features=300, out_features=300, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): Linear(in_features=300, out_features=1, bias=True)
  )
)

In [46]:
sum(p.numel() for p in model.parameters())

354901

In [47]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
scheduler = NoamLR(
    optimizer,
    warmup_epochs=[2],
    total_epochs=[num_epochs],
    steps_per_epoch=len(train_loader),
    init_lr=[1e-4],
    max_lr=[1e-3],
    final_lr=[1e-4],
)

In [48]:
dataset.num_node_features, dataset.num_edge_features

(133, 14)

In [49]:
config = {
    "loss": criterion,
    "model": model,
    "optimizer": optimizer,
    "scheduler": scheduler,
}

In [50]:
best = {'epoch': -1, 'score': float("-inf")}
outfile = f"{data_dir}/best_snapshot.pth"

In [51]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}", file=sys.stderr)
    train(config, train_loader, device=cuda)
    y_pred, y_true = make_prediction(config, valid_loader, device=cuda)
    auc = roc_auc_score(y_true, y_pred)
    if auc > best['score']:
        print(f"* best-auc {best['score']:.6} ==> {auc:.6}", file=sys.stderr)
        best['score'] = auc
        best['epoch'] = epoch+1
        torch.save(model.state_dict(), outfile)
    acc = accuracy_score(y_true, (y_pred > 0.5).int())
    print(f"val auc={auc:.6} acc={acc:.6}", file=sys.stderr)

Epoch 1
100%|██████████| 83/83 [00:00<00:00, 181.39it/s]
* best-auc -inf ==> 0.716913
val auc=0.716913 acc=0.980306
Epoch 2
100%|██████████| 83/83 [00:00<00:00, 182.14it/s]
* best-auc 0.716913 ==> 0.760104
val auc=0.760104 acc=0.978118
Epoch 3
100%|██████████| 83/83 [00:00<00:00, 179.39it/s]
val auc=0.717173 acc=0.981765
Epoch 4
100%|██████████| 83/83 [00:00<00:00, 148.55it/s]
val auc=0.736763 acc=0.981765
Epoch 5
100%|██████████| 83/83 [00:01<00:00, 79.27it/s] 
* best-auc 0.760104 ==> 0.771881
val auc=0.771881 acc=0.980549
Epoch 6
100%|██████████| 83/83 [00:00<00:00, 168.66it/s]
val auc=0.756139 acc=0.981765
Epoch 7
100%|██████████| 83/83 [00:00<00:00, 162.93it/s]
* best-auc 0.771881 ==> 0.783118
val auc=0.783118 acc=0.982008
Epoch 8
100%|██████████| 83/83 [00:00<00:00, 149.28it/s]
* best-auc 0.783118 ==> 0.792763
val auc=0.792763 acc=0.982251
Epoch 9
100%|██████████| 83/83 [00:00<00:00, 146.30it/s]
* best-auc 0.792763 ==> 0.794811
val auc=0.794811 acc=0.981522
Epoch 10
100%|█████████

In [52]:
print(f"best validation auc = {best['score']} on epoch {best['epoch']}")

best validation auc = 0.8118539339604155 on epoch 20


#### final 

In [53]:
y_pred, y_true = make_prediction(config, test_loader, device=cuda)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"test auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 83/83 [00:00<00:00, 169.93it/s]
test auc=0.762168 acc=0.961585


#### best snapshot

In [54]:
model = nn.Sequential(
    DMPNNEncoder(
        hidden_size,
        dataset.num_node_features,
        dataset.num_edge_features,
        depth,
    ),
    head,
)
model.load_state_dict(torch.load(outfile))

<All keys matched successfully>

In [55]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
scheduler = NoamLR(
    optimizer,
    warmup_epochs=[2],
    total_epochs=[num_epochs],
    steps_per_epoch=len(train_loader),
    init_lr=[1e-4],
    max_lr=[1e-3],
    final_lr=[1e-4],
)

In [56]:
config = {
    "loss": criterion,
    "model": model,
    "optimizer": optimizer,
    "scheduler": scheduler,
}

In [57]:
y_pred, y_true = make_prediction(config, train_loader, device=cuda)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"train auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 659/659 [00:03<00:00, 173.13it/s]
train auc=0.97518 acc=0.978329


In [58]:
y_pred, y_true = make_prediction(config, valid_loader, device=cuda)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"valid auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 83/83 [00:00<00:00, 173.54it/s]
valid auc=0.811854 acc=0.980793


In [59]:
y_pred, y_true = make_prediction(config, test_loader, device=cuda)
auc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, (y_pred > 0.5).int())
print(f"test auc={auc:.6} acc={acc:.6}", file=sys.stderr)

100%|██████████| 83/83 [00:00<00:00, 148.81it/s]
test auc=0.784245 acc=0.963773


#### performances of other models

https://ogb.stanford.edu/docs/leader_graphprop/#ogbg-molhiv

#### original chemprop

```bash
$ python train.py --data_path ../train_hiv.csv --separate_val_path ../valid_hiv.csv --separate_test_path ../test_hiv.csv --target_columns HIV_active --smiles_columns smiles --dataset_type classification --save_dir tmp --explicit_h --seed 0 --quiet
```

```bash
Seed 0 ==> test auc = 0.794824
Seed 1 ==> test auc = 0.788354
Seed 2 ==> test auc = 0.772676
```

```python
MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout_layer): Dropout(p=0.0, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=300, bias=False)
        (W_h): Linear(in_features=300, out_features=300, bias=False)
        (W_o): Linear(in_features=433, out_features=300, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): Linear(in_features=300, out_features=300, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): Linear(in_features=300, out_features=1, bias=True)
  )
)
Number of parameters = 355,201
```

**Note: this # of parameters includes "cached_zero_vector" (300)** and 354901 = 355201-300

In [60]:
layers = [
    nn.Dropout(p=0.0, inplace=False),
    nn.ReLU(),
    nn.Linear(in_features=147, out_features=300, bias=False),
    nn.Linear(in_features=300, out_features=300, bias=False),
    nn.Linear(in_features=433, out_features=300, bias=True),
    nn.Dropout(p=0.0, inplace=False),
    nn.Linear(in_features=300, out_features=300, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.0, inplace=False),
    nn.Linear(in_features=300, out_features=1, bias=True)
]
num_params = 0
for l in layers:
    num_params += sum(p.numel() for p in l.parameters())
num_params

354901

```bash
Model 0 best validation auc = 0.819074 on epoch 4
Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda
Model 0 test auc = 0.794824                                                                                                 
Ensemble test auc = 0.794824
1-fold cross validation
        Seed 0 ==> test auc = 0.794824
Overall test auc = 0.794824 +/- 0.000000
Elapsed time = 0:06:50
```